In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession \
    .builder \
    .appName("DataCleaner") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/ECE552FinalProject.TrafficData") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/ECE552FinalProject.TrafficData") \
    .getOrCreate()

In [ ]:
#Read in parquet
parquetDF = spark.read.parquet("TrafficAccidentsData.parquet")

In [ ]:
states_abbrev = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

In [ ]:
pipeline = "[{ $group : { _id : '$State'}}]"
#[{$group:{_id: "$State"}}]
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("pipeline", pipeline).load()
print(df.count())
#This does not work as intended as spark is trying to do this job with multiple partitions of the data in mongo, 
#returning multiple identical tags it appears, if you do this exact pipeline command in mongoDB compass, it works fine
#have to do some extra aggregation in spark

In [ ]:
%%time
pipeline = "[{ $group : { _id : '$Astronomical_Twilight', count:{$sum:1}}}, {$sort:{_id:1}}]"
df = spark.read.format("mongo").option("pipeline", pipeline).load()
df = df.groupby("_id").sum()
df = df.withColumnRenamed("_id","AccidentAstronomicalTwilight").withColumnRenamed("sum(count)","AccidentCount")
df = df.orderBy("AccidentAstronomicalTwilight")
print(df.count())
df.agg({'AccidentCount': 'sum'}).show()
df.show(49)

In [ ]:
df.write.format("mongo").option("collection", "AccidentCountByAstronomicalTwilight").mode("overwrite").save()